In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 3, 19)

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trade name qty target active reason market xdate'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,21,S,AIMIRT,10000,13.1,0,DOS,SET,2022-03-14


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate
0,B,DCC,-8,6L,SET,45000,2.80,2.96,-0.02,-0.67%,0,2022-04-05
1,B,MAKRO,-6,RD05pct,SET,1500,39.75,41.25,-0.25,-0.60%,0,2022-03-03
2,B,RCL,-9,DOS,SET100,3000,48.25,50.50,-0.75,-1.46%,0,2022-03-23
3,B,STA,-4,RD20pct,SET100,15000,26.25,27.25,-0.75,-2.68%,0,2022-04-19
4,B,TISCO,-7,ROUND,SET50,500,97.75,99.50,-0.50,-0.50%,0,2022-04-29
5,B,TMT,-7,ROUND,SET,15000,10.50,11.20,0.00,0.00%,0,2022-04-18
6,S,AIMIRT,2,DOS,SET,10000,13.10,12.90,0.00,0.00%,0,2022-03-14
7,S,ASP,17,DOS,SET,30000,3.82,3.48,0.00,0.00%,0,2022-03-10
8,S,BCH,2,CP1S,SET100,5000,21.40,21.20,0.20,+0.95%,0,2022-05-10
9,S,CPNCG,2,DOS,SET,20000,12.50,12.30,0.00,0.00%,0,2022-02-14


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0     4
1    17
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

21

In [9]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market,xdate
0,B,DCC,45000,2.80,1,6L,SET,2022-04-05
1,B,MAKRO,1500,39.75,1,RD05pct,SET,2022-03-03
2,B,RCL,3000,48.25,1,DOS,SET100,2022-03-23
3,B,STA,15000,26.25,1,RD20pct,SET100,2022-04-19
4,B,TISCO,500,97.75,1,ROUND,SET50,2022-04-29
5,B,TMT,15000,10.50,1,ROUND,SET,2022-04-18
6,S,AIMIRT,10000,13.10,1,DOS,SET,2022-03-14
7,S,ASP,30000,3.82,0,DOS,SET,2022-03-10
8,S,BCH,5000,21.40,1,CP1S,SET100,2022-05-10
9,S,CPNCG,20000,12.50,1,DOS,SET,2022-02-14


In [10]:
rcds = final.values.tolist()
rcds

[['B', 'DCC', 45000, 2.8, 1, '6L', 'SET', '2022-04-05'],
 ['B', 'MAKRO', 1500, 39.75, 1, 'RD05pct', 'SET', '2022-03-03'],
 ['B', 'RCL', 3000, 48.25, 1, 'DOS', 'SET100', '2022-03-23'],
 ['B', 'STA', 15000, 26.25, 1, 'RD20pct', 'SET100', '2022-04-19'],
 ['B', 'TISCO', 500, 97.75, 1, 'ROUND', 'SET50', '2022-04-29'],
 ['B', 'TMT', 15000, 10.5, 1, 'ROUND', 'SET', '2022-04-18'],
 ['S', 'AIMIRT', 10000, 13.1, 1, 'DOS', 'SET', '2022-03-14'],
 ['S', 'ASP', 30000, 3.82, 0, 'DOS', 'SET', '2022-03-10'],
 ['S', 'BCH', 5000, 21.4, 1, 'CP1S', 'SET100', '2022-05-10'],
 ['S', 'CPNCG', 20000, 12.5, 1, 'DOS', 'SET', '2022-02-14'],
 ['S', 'DOHOME', 4800, 23.3, 1, '15pct', 'SET100', '2022-02-02'],
 ['S', 'HREIT', 40100, 9.0, 1, 'DOS', 'SET', '2022-03-04'],
 ['S', 'IP', 2500, 20.3, 0, 'CP1S', 'mai', '2022-02-02'],
 ['S', 'JASIF', 10000, 11.3, 1, 'DOS', 'SET', '2022-03-03'],
 ['S', 'KCE', 1000, 68.0, 0, 'CUT', 'SET50', '2022-03-21'],
 ['S', 'PTT', 3000, 41.25, 1, '5pct', 'SET50', '2022-03-03'],
 ['S', 'RATCH

In [11]:
for rcd in rcds:
    print(rcd)

['B', 'DCC', 45000, 2.8, 1, '6L', 'SET', '2022-04-05']
['B', 'MAKRO', 1500, 39.75, 1, 'RD05pct', 'SET', '2022-03-03']
['B', 'RCL', 3000, 48.25, 1, 'DOS', 'SET100', '2022-03-23']
['B', 'STA', 15000, 26.25, 1, 'RD20pct', 'SET100', '2022-04-19']
['B', 'TISCO', 500, 97.75, 1, 'ROUND', 'SET50', '2022-04-29']
['B', 'TMT', 15000, 10.5, 1, 'ROUND', 'SET', '2022-04-18']
['S', 'AIMIRT', 10000, 13.1, 1, 'DOS', 'SET', '2022-03-14']
['S', 'ASP', 30000, 3.82, 0, 'DOS', 'SET', '2022-03-10']
['S', 'BCH', 5000, 21.4, 1, 'CP1S', 'SET100', '2022-05-10']
['S', 'CPNCG', 20000, 12.5, 1, 'DOS', 'SET', '2022-02-14']
['S', 'DOHOME', 4800, 23.3, 1, '15pct', 'SET100', '2022-02-02']
['S', 'HREIT', 40100, 9.0, 1, 'DOS', 'SET', '2022-03-04']
['S', 'IP', 2500, 20.3, 0, 'CP1S', 'mai', '2022-02-02']
['S', 'JASIF', 10000, 11.3, 1, 'DOS', 'SET', '2022-03-03']
['S', 'KCE', 1000, 68.0, 0, 'CUT', 'SET50', '2022-03-21']
['S', 'PTT', 3000, 41.25, 1, '5pct', 'SET50', '2022-03-03']
['S', 'RATCH', 6000, 46.0, 1, 'DOS', 'SET50',

In [12]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [13]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [14]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [15]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)